In [2]:
#hide
# default_exp script

# fastscript

> A fast way to turn your python function into a script.

Part of [fast.ai](https://www.fast.ai)'s toolkit for delightful developer experiences.

## Install

Either `pip install fastscript` or `conda install -c fastai fastscript`

## Overview

Sometimes, you want to create a quick script, either for yourself, or for others. But in Python, that involves a whole lot of boilerplate and ceremony, especially if you want to support command line arguments, provide help, and other niceties. You can use [argparse](https://docs.python.org/3/library/argparse.html) for this purpose, which comes with Python, but it's complex and verbose.

`fastscript` makes life easier. There are much fancier modules to help you write scripts (we recommend [Python Fire](https://github.com/google/python-fire), and [Click](https://click.palletsprojects.com/en/7.x/) is also popular), but fastscript is very fast and very simple. In fact, it's <50 lines of code! Basically, it's just a little wrapper around `argparse` that uses modern Python features and some thoughtful defaults to get rid of the boilerplate.

For full details, see the [docs](https://fastscript.fast.ai) for `core`.

## Example

Here's a complete example:

```python
from fastscript import *
@call_parse
def main(msg:Param("The message", str),
         upper:Param("Convert to uppercase?", bool_arg)=False):
    print(msg.upper() if upper else msg)
````

If you copy that info a file and run it, you'll see:

```
$ python examples/test_fastscript.py
usage: test_fastscript.py [-h] [--upper UPPER] msg
test_fastscript.py: error: the following arguments are required: msg
```

As you see, we didn't need any `if __name__ == "__main__"`, we didn't have to parse arguments, we just wrote a function, added a decorator to it, and added some annotations to our function's parameters. As a bonus, we can also use this function directly from a REPL such as Jupyter Notebook - it's not just for command line scripts!

## Param annotations

Each parameter in your function should have an annotation `Param(...)` (as in the example above). You can pass the following when calling `Param`: `help`,`type`,`opt`,`action`,`nargs`,`const`,`choices`,`required` . Except for `opt`, all of these are just passed directly to `argparse`, so you have all the power of that module at your disposal. Generally you'll want to pass at least `help` (since this is provided as the help string for that parameter) and `type` (to ensure that you get the type of data you expect). `opt` is a bool that defines whether a param is optional or required (positional) - but you'll generally not need to set this manually, because fastscript will set it for you automatically based on *default* values.

You should provide a default (after the `=`) for any *optional* parameters. If you don't provide a default for a parameter, then it will be a *positional* parameter.

## setuptools scripts

There's a really nice feature of pip/setuptools that lets you create commandline scripts directly from functions, makes them available in the `PATH`, and even makes your scripts cross-platform (e.g. in Windows it creates an exe). fastscript supports this feature too. To use it, follow [this example](fastscript/test_cli.py) from `fastscript/test_cli.py` in the repo. As you see, it's basically identical to the script example above, except that we can treat it as a module. The trick to making this available as a script is to add a `console_scripts` section to your setup file, of the form: `script_name=module:function_name`. E.g. in this case we use: `test_fastscript=fastscript.test_cli:main`. With this, you can then just type `test_fastscript` at any time, from any directory, and your script will be called (once it's installed using one of the methods below).

You don't actually have to write a `setup.py` yourself. Instead, just use [nbdev](https://nbdev.fast.ai). Then modify `settings.ini` as appropriate for your module/script. To install your script directly, you can type `pip install -e .`. Your script, when installed this way (it's called an [editable install](http://codumentary.blogspot.com/2014/11/python-tip-of-year-pip-install-editable.html), will automatically be up to date even if you edit it - there's no need to reinstall it after editing. With nbdev you can even make your module and script available for installation directly from pip and conda by running `make release`.

## API details

In [11]:
from fastcore.test import *

In [24]:
#export
import inspect,functools
import argparse
from fastcore.imports import *

In [13]:
#export
class Param:
    "A parameter in a function used in `anno_parser` or `call_parse`"
    def __init__(self, help=None, type=None, opt=True, action=None, nargs=None, const=None,
                 choices=None, required=None):
        self.help,self.type,self.opt,self.action,self.nargs = help,type,opt,action,nargs
        self.const,self.choices,self.required = const,choices,required
        
    def set_default(self, d):
        if d==inspect.Parameter.empty: self.opt = False
        else:
            self.default = d
            self.help += f" (default: {d})"

    @property
    def pre(self): return '--' if self.opt else ''
    @property
    def kwargs(self): return {k:v for k,v in self.__dict__.items() if v is not None and k!='opt'}

Each parameter in your function should have an annotation `Param(...)`. You can pass the following when calling `Param`: `help`,`type`,`opt`,`action`,`nargs`,`const`,`choices`,`required` (i.e. it takes the same parameters as `argparse.ArgumentParser.add_argument`, plus `opt`). Except for `opt`, all of these are just passed directly to `argparse`, so you have all the power of that module at your disposal. Generally you'll want to pass at least `help` (since this is provided as the help string for that parameter) and `type` (to ensure that you get the type of data you expect).

`opt` is a bool that defines whether a param is optional or required (positional) - but you'll generally not need to set this manually, because fastscript will set it for you automatically based on *default* values. You should provide a default (after the `=`) for any *optional* parameters. If you don't provide a default for a parameter, then it will be a *positional* parameter.

In [14]:
p = Param(help="help", type=int)
p.set_default(1)
test_eq(p.kwargs, {'help': 'help (default: 1)', 'type': int, 'default': 1})

In [15]:
#export
def bool_arg(v):
    "Use as `type` for `Param` to get `bool` behavior"
    if isinstance(v, bool): return v
    if v.lower() in ('yes', 'true', 't', 'y', '1'): return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'): return False
    else: raise argparse.ArgumentTypeError('Boolean value expected.')

In [16]:
def f(from_name:Param("Get args from prog name instead of argparse", bool_arg)=0,
      a:Param("param 1", bool_arg)=1,
      b:Param("param 2", str)="test"): ...

In [17]:
#export
def anno_parser(func, prog=None, from_name=False):
    "Look at params (annotated with `Param`) in func and return an `ArgumentParser`"
    p = argparse.ArgumentParser(description=func.__doc__, prog=prog)
    for k,v in inspect.signature(func).parameters.items():
        param = func.__annotations__.get(k, Param())
        param.set_default(v.default)
        p.add_argument(f"{param.pre}{k}", **param.kwargs)
    p.add_argument(f"--xtra", help="Parse for additional args", type=str)
    return p

This converts a function with parameter annotations of type `Param` into an `argparse.ArgumentParser` object. Function arguments with a default provided are optional, and other arguments are positional.

In [18]:
def f(required:Param("Required param", int),
      a:Param("param 1", bool_arg)=1,
      b:Param("param 2", str)="test"):
    "my docs"
    ...

p = anno_parser(f, 'progname')
p.print_help()

usage: progname [-h] [--a A] [--b B] [--xtra XTRA] required

my docs

positional arguments:
  required     Required param

optional arguments:
  -h, --help   show this help message and exit
  --a A        param 1 (default: 1)
  --b B        param 2 (default: test)
  --xtra XTRA  Parse for additional args


In [19]:
#export
def args_from_prog(func, prog):
    "Extract args from `prog`"
    if '##' in prog: _,prog = prog.split('##', 1)
    progsp = prog.split("#")
    args = {progsp[i]:progsp[i+1] for i in range(0, len(progsp), 2)}
    for k,v in args.items():
        t = func.__annotations__.get(k, Param()).type
        if t: args[k] = t(v)
    return args

Sometimes it's convenient to extract arguments from the actual name of the called program. `args_from_prog` will do this, assuming that names and values of the params are separated by a `#`. Optionally there can also be a prefix separated by `##` (double underscore).

In [20]:
exp = {'a': False, 'b': 'baa'}
test_eq(args_from_prog(f, 'foo##a#0#b#baa'), exp)
test_eq(args_from_prog(f, 'a#0#b#baa'), exp)

In [21]:
#export
def call_parse(func):
    "Decorator to create a simple CLI from `func` using `anno_parser`"
    mod = inspect.getmodule(inspect.currentframe().f_back)
    if not mod: return func
    
    @functools.wraps(func)
    def _f(*args, **kwargs):
        mod = inspect.getmodule(inspect.currentframe().f_back)
        if not mod: return func(*args, **kwargs)
    
        p = anno_parser(func)
        args = p.parse_args()
        xtra = getattr(args, 'xtra', None)
        if xtra is not None:
            if xtra==1: xtra = p.prog
            for k,v in args_from_prog(func, xtra).items(): setattr(args,k,v)
        del(args.xtra)
        func(**args.__dict__)
    if mod.__name__=="__main__":
        setattr(mod, func.__name__, _f)
        return _f()
    else: return _f

In [22]:
@call_parse
def test_add(a:Param("param a", int), b:Param("param 1",int)):
    return a + b

### Test call parse function

`call_parse` decorated functions work as regular functions and also as command-line interface functions.

In [25]:
test_eq(os.system('test_fastscript "Test if this still works"'), 0)
test_eq(test_add(1,2), 3)

This is the main way to use `fastscript`; decorate your function with `call_parse`, add `Param` annotations as shown above, and it can then be used as a script.

## Export -

In [26]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_test.ipynb.
Converted 01_foundation.ipynb.
Converted 02_utils.ipynb.
Converted 03_dispatch.ipynb.
Converted 04_transform.ipynb.
Converted 05_logargs.ipynb.
Converted 06_meta.ipynb.
Converted 07_script.ipynb.
Converted 08_tmp.ipynb.
Converted index.ipynb.
